# Lab 2: Backpropagation for NXOR
---

Grupo 3 \
Alexandre Rodrigues: 75545 \
Tiago Granja: 79845 \
Diogo Silva: 79828

Import libraries

In [1]:
import numpy as np

---
1.

To calculate the weights update we first need to feedforward our network. With our 2:1 architecture we have 6 weights, 4 in the hidden layer, $w_{11}^{[1]}, w_{21}^{[1]}, w_{12}^{[1]}, w_{22}^{[1]}$, and 2 in the output layer, $w_1^{[2]}, w_2^{[2]}$, and 3 biases, $b_1^{[1]}, b_2^{[1]}$, in the first layer and $b_1^{[2]}$ in the second. With this we can write our inner weighted sums and outputs as follows:

$$
\begin{split}
z_1^{[1]} = x_1 w_{11}^{[1]} + x_2 w_{21}^{[1]} + b_1^{[1]} \\
y_1^{[1]} = S(z_1^{[1]}) \\
\\
z_2^{[1]} = x_1 w_{12}^{[1]} + x_2 w_{22}^{[1]} + b_2^{[1]} \\
y_2^{[1]} = S(z_2^{[1]}) \\
\\
z_1^{[2]} = y_1^{[1]} w_1^{[2]} + y_2^{[1]} w_2^{[2]} + b_1^{[2]} \\
y = S(z_1^{[2]})
\end{split}
$$

Where S is the sigmoid function.

#### Backpropagation

For backpropagation we use the squared error as our error where $E = \frac{1}{2}(y - t)^2$, where $t$ is our target output. With this we can start writing the expressions for the weight updates such that:

$$
\begin{split}
w_{ij}^{[k]} = w_{ij}^{[k]} - \upeta \frac{\partial E}{\partial w_{ij}^{[k]}}
\end{split}
$$

##### Outer layer

For the outer layer we can write:

$$
\begin{split}
\Delta w_{i}^{[2]} = \frac{\partial E}{\partial w_{i}^{[2]}} \\
\Delta w_{i}^{[2]} = \frac{\partial E}{\partial y}\frac{\partial y}{\partial z_1^{[2]}}\frac{\partial z_1^{[2]}}{\partial w_i^{[2]}} \\
\Delta w_{i}^{[2]} = (y - t) S'(z_1^{[2]})y_i^{[1]} \\
\end{split}
$$

Where the last term is equal to one when taking the partial derivative in order to $b_1^{[2]}$ and $S'$ is the derivative of $S$.

##### Inner (Hidden) layer

For the inner layer we can write:

$$
\begin{split}
\Delta w_{ij}^{[1]} = \frac{\partial E}{\partial w_{ij}^{[1]}} \\
\Delta w_{ij}^{[1]} = \frac{\partial E}{\partial y_j^{[1]}}\frac{\partial y_j^{[1]}}{\partial z_j^{[1]}}\frac{\partial z_j^{[1]}}{\partial w_{ij}^{[1]}} \\
\Delta w_{ij}^{[1]} = \frac{\partial E}{\partial y_j^{[1]}} S'(z_j^{[1]}) x_i
\end{split}
$$

Where:

$$
\begin{split}
\frac{\partial E}{\partial y_j^{[1]}} = \frac{\partial E}{\partial y}\frac{\partial y}{\partial z_1^{[2]}}\frac{\partial z_1^{[2]}}{\partial y_j^{[1]}} \\
\frac{\partial E}{\partial y_j^{[1]}} = (y - t) S'(z_1^{[2]}) w_j^{[2]}
\end{split}
$$

Concluding:

$$
\begin{split}
\Delta w_{ij}^{[1]} = (y - t) S'(z_1^{[2]}) w_j^{[2]} S'(z_j^{[1]}) x_i
\end{split}
$$

Where the last term is equal to one when taking the partial derivative in order to $b_1^2$ and $S'$ is the derivative of $S$.

---
2.

In [43]:
def sigmoid(x: float) -> float:
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x: float) -> float:
    return sigmoid(x) * (1 - sigmoid(x))

In [46]:
class BinaryTwoOneNetwork:

    def __init__(self, learning_rate: float):
        self._learning_rate = learning_rate
        
        self._inner_weights = np.random.uniform(0.0, 0.1, (2, 2))
        self._inner_biases = np.random.uniform(0.0, 0.1, 2)
        self._outer_weights = np.random.uniform(0.0, 0.1, 2)
        self._outer_bias = np.random.uniform(0.0, 0.1)

        self._inner_sums = np.empty(2)
        self._inner_outputs = np.empty(2)
    
    def _feed_forward(self, x1: float, x2:float) -> float:

        # Inner layer
        self._inner_sums[0] = self._inner_weights[0][0] * x1 + self._inner_weights[1][0] * x2 + self._inner_biases[0]
        self._inner_outputs[0] = sigmoid(self._inner_sums[0])
        self._inner_sums[1] = self._inner_weights[0][1] * x1 + self._inner_weights[1][1] * x2 + self._inner_biases[1]
        self._inner_outputs[1] = sigmoid(self._inner_sums[1])

        # Outer layer
        self._outer_sum = self._outer_weights[0] * self._inner_outputs[0] + self._outer_weights[1] * self._inner_outputs[1] + self._outer_bias
        self._output = sigmoid(self._outer_sum)

        return self._output

    def _backpropagate(self, x1: float, x2:float, target: float) -> None:
        error = (self._output - target)
        delta_output = error * sigmoid_derivative(self._outer_sum)
        
        # Inner layer
        for j in range(2):
            self._inner_weights[0][j] -= self._learning_rate * delta_output * self._outer_weights[j] * sigmoid_derivative(self._inner_sums[j]) * x1
            self._inner_weights[1][j] -= self._learning_rate * delta_output * self._outer_weights[j] * sigmoid_derivative(self._inner_sums[j]) * x2
            self._inner_biases[j] -= self._learning_rate * delta_output * self._outer_weights[j] * sigmoid_derivative(self._inner_sums[j])

        # Outer layer
        for i in range(2):
            self._outer_weights[i] -= self._learning_rate * delta_output * self._inner_outputs[i]
        self._outer_bias -= self._learning_rate * delta_output

In [45]:
x = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

target = np.array([[1],
                     [0],
                     [0],
                     [1]])

network = BinaryTwoOneNetwork(0.1)
iterations = 1000
for i in range(iterations):
    for j in range(4):
        network._feed_forward(x[j][0], x[j][1])
        network._backpropagate(x[j][0], x[j][1], target[j][0])

for j in range(4):
    print(network._feed_forward(x[j][0], x[j][1]))

0.49994585414077714
0.500083575348709
0.4999759092988594
0.50011359519794
